> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# Modelagem de dados
Python é uma linguagem muito permissiva, isso é fato. Mas no escopo de ciência de dados, e em geral quando interagimos com algum sistema de gestão de banco de dados (SGBD), é necessário ser mais restritivo quanto ao tipo e à natureza dos dados para garantir **integridade de dados** (garantir precisão, completude, consistência e validade dos dados de uma organização).

Impor restrições de tipo, além de realizar validações robustas, é fundamental para assegurar a integridade. Restrições de tipo garantem que os dados sejam do formato esperado, prevenindo erros e inconsistências. Validações robustas asseguram que os dados cumpram regras específicas de negócio e lógicas, evitando a entrada de dados inválidos que poderiam corromper o banco de dados ou comprometer a análise.

Nas próximas aulas conheceremos alguns recursos do Python para trabalhar com integridade de dados em mente.


## Type hints (anotações de tipo)

Referências principalis:
* [PEP 484 - Type Hints](https://peps.python.org/pep-0484/)
* [Type Hint Cheat Sheet](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html)

Type hinting no Python é uma funcionalidade que permite aos desenvolvedores especificar o tipo de variáveis, parâmetros de função e valores de retorno. Por padrão, são somente para indicar aos desenvolvedores os tipos esperados em determinados contextos, no entanto **Python é uma linguagem dinamicamente tipada**, e sua execução continua inalterada. Ou seja, **type hints não adicionam verificações de tipo à execução**.
> Veremos mais à frente que existem outras bibliotecas (como a Pydantic) que exploram a anotação de tipo para realizar validações.

Sintaxe:
* **Variáveis**: Você pode especificar o tipo de uma variável usando a notação `:`
* **Métodos e funções**: Para funções, você pode especificar os tipos dos parâmetros com a notação `:` e os valores de retorno com a notação `->`.

In [ ]:
def divide(a: int, b: int) -> float:
    return a/b

def inverter(st: str) -> str:
    return st[::-1]

def cumprimentar(nome: str) -> None:
    print(f'Olá {nome}!')


x: int = 10
y: int = 17
res = divide(x, y)
print(f'A soma {x} + {y} = {res:.2}')

nome: str = "Mila"
cumprimentar(nome)
print(inverter(nome))

A soma 10 + 17 = 0.59
Olá Mila!
aliM


O atributo `__annotations__` registra todos os elementos de um dado objeto que são acompanhados de anotação de tipo.

In [ ]:
print(divide.__annotations__)

{'a': <class 'int'>, 'b': <class 'int'>, 'return': <class 'float'>}


No caso de coleções, podemos indicar não só o tipo da coleção como também o tipo de seus elementos. Usa-se nesse caso a sintaxe de colchetes, como apresentado a seguir.

In [ ]:
def soma_lista(lista: list[int]) -> int:
    return sum([x * 2 for x in lista])

def soma_dicionario(d: dict[str, int]) -> int:
    return sum(d.values())

def soma_tupla(lista: list[tuple[int, int]]) -> tuple[int, int]:
    return (sum(v[0] for v in lista), sum(v[1] for v in lista))

lst: list[int] = [1,2,3]
print(soma_lista(lst))

dic: dict[str, int] = {'a': 2, 'b':3, 'c': 4}
print(soma_dicionario(dic))

tup: list[tuple[int, int]] = [(1,2), (3,4), (5,6)]
print(soma_tupla(tup))

12
9
(9, 12)


### `typing`

A biblioteca `typing` expande nossa capacidade de anotação de tipos. Por exemplo, no caso de parâmetros opcionais você pode usar a anotação `Optional`. Assim como no caso de coleções, indicamos com a sintaxe de colchetes o tipo esperado para a variável opcional.

> Lembre-se que a anotação de tipo não altera a execução do Python e portanto é preciso definir valores padrão para variáveis opcionais. Caso contrário, elas serão consideradas obrigatórias.

In [ ]:
from typing import Optional, Any

def imprime_lista(qualquer_coisa: list[Any]) -> None:
    print(qualquer_coisa)

def cumprimentar(nome: Optional[str] = None) -> None:
    if nome:
        print(f'Olá {nome}!')
    else:
        print('Olá mundo!')

cumprimentar()

Olá mundo!


Em alguns casos precisaremos anotar uma dada função ou variável com **tipos genéricos**, para indicar que aquele elemento pode assumir mais de um tipo de dado. No Python, você pode usar a biblioteca `typing`para definir tipos genéricos.

* `Sequence` é um dos [duck types](https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html#standard-duck-types) disponíveis para coleções de valores (*lembra? Se anda como um pato...*). Ao anotar um tipo `Sequence`, assumimos qualquer tipo de dados que possui os métodos mágicos `len` e `__getitem__`.

* `TypeVar` por outro lado nos permite garantir consistência de tipos, sem estabelecer estáticamente um tipo exato. Por exemplo, na função `first` a seguir indicamos que o valor retornado tem o mesmo tipo dos elementos da sequência.
    * Uma expressão `TypeVar()` deve sempre ser atribuída diretamente a uma variável. O primeiro argumento de `TypeVar()` deve ser uma string igual ao nome da variável à qual ela é atribuída.
    * Podemos adicionar mais parâmetros a `TypeVar()` para restringir o conjunto de tipos aceitos no determinado escopo.

In [ ]:
from typing import Sequence, TypeVar

# Anotação genérica de tipo
T = TypeVar('T')
def first(l: list[T]) -> T:
    return l[0]

# Tipo genérico de sequência
T = TypeVar('T')
def first(l: Sequence[T]) -> T:
    return l[0]

# Anotação genérica de tipo
# restrita a str ou bytes
AnyStr = TypeVar('AnyStr', str, bytes)
def concat(x: AnyStr, y: AnyStr) -> AnyStr:
    return x + y

## `dataclass`

O módulo `dataclasses` fornece alguns decoradores e funções que adicionam automaticamente alguns métodos especiais (mágicos ✨🪄🔮) a classes customizadas (que a gente mesmo cria).

A principal função é a `dataclass`, típicamente usada como decorador de classe. Seu papel é aplicar um pós-processamento à classe decorada, examinando-a para encontrar campos (*fields*). Um campo (*field*) é definido como qualquer variável identificada em `__annotations__`. Ou seja, uma variável que tem uma anotação de tipo (*type hint*).

Você pode fornecer parâmetros ao `dataclass` para indicar quais métodos serão automaticamente adicionados. No exemplo a seguir, as duas decoração são equivalentes. O segundo caso apenas apresenta todas as customizações disponíveis associadas a seus valores padrão.


```python
from dataclasses import dataclass

@dataclass
class A:
    field1: str
    field2: int = 0
    ...

@dataclass(init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
class B:
    field1: str
    field2: int = 0
```

A seguir uma explicação dos principais parâmetros (você pode consultar os outros na [PEP-557](https://peps.python.org/pep-0557/)).

* `init`: Se `True` (o padrão), um método `__init__` será gerado.
* `repr`: Se `True` (o padrão), um método `__repr__` será gerado. A string repr terá o nome da classe e o nome e repr de cada campo, na ordem em que são definidos na classe.
* `eq`: Se `True` (o padrão), um método `__eq__` será gerado. Este método compara a classe como se fosse uma tupla de seus campos, em ordem.
* `order`: Se `True` (o padrão é `False`), os métodos `__lt__`, `__le__`, `__gt__` e `__ge__` serão gerados. Eles comparam a classe como se fosse uma tupla de seus campos, em ordem. Se `order` for `True`, `eq` também deve ser `True`.

Para o exemplo definido na célula a seguir, segue uma amostra de como fica sua estrutura interna em termos de métodos mágicos.
```python
def __init__(self, nome: str, preco_unitario: float, quantidade_em_estoque: int = 0) -> None:
    self.nome = nome
    self.preco_unitario = preco_unitario
    self.quantidade_em_estoque = quantidade_em_estoque
def __repr__(self):
    return f'InventoryItem(nome={self.nome!r}, preco_unitario={self.preco_unitario!r}, quantidade_em_estoque={self.quantidade_em_estoque!r})'
def __eq__(self, other):
    if other.__class__ is self.__class__:
        return (self.nome, self.preco_unitario, self.quantidade_em_estoque) == (other.nome, other.preco_unitario, other.quantidade_em_estoque)
    return NotImplemented
def __lt__(self, other):
    if other.__class__ is self.__class__:
        return (self.nome, self.preco_unitario, self.quantidade_em_estoque) < (other.nome, other.preco_unitario, other.quantidade_em_estoque)
    return NotImplemented
    ... # outros métodos a seguir ...
```

In [ ]:
from dataclasses import dataclass

@dataclass
class ItemDeInventario:
    '''Classe para registrar um item no inventário.'''
    nome: str
    preco_unitario: float
    quantidade_em_estoque: int = 0

    def custo_total(self) -> float:
        return self.preco_unitario * self.quantidade_em_estoque

print(ItemDeInventario.__annotations__)

item = ItemDeInventario("", 0)
dir(item)

{'nome': <class 'str'>, 'preco_unitario': <class 'float'>, 'quantidade_em_estoque': <class 'int'>}


['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'custo_total',
 'nome',
 'preco_unitario',
 'quantidade_em_estoque']

### `field`

Também podemos definir e customizar explícitamente cada um dos campos de dados. Para isso, temos a função `field` do módulo `dataclasses`, com a seguite assinatura.

```python
def field(*, default=MISSING, default_factory=MISSING, repr=True,
          hash=None, init=True, compare=True, metadata=None)
```

Os principais parâmetros para `field()` estão a seguir. Você pode consultar a lista completa [na documentação](https://docs.python.org/3/library/dataclasses.html#dataclasses.field).

* `default`: Se fornecido, este será o valor padrão do campo. Isso é necessário porque a chamada `field()` substitui a posição normal do valor padrão.

* `default_factory`: Se fornecido, deve ser um objeto `callable` (ex.: função ou classe) sem parâmetros que será chamado quando um valor padrão for necessário para este campo. Pode ser usado para especificar campos com valores padrão mutáveis. É um erro especificar `default` e `default_factory`.

* `init`: Se `True` (o padrão), este campo é incluído como um parâmetro para o método `__init__()` gerado.

* `repr`: Se `True` (o padrão), este campo é incluído na string retornada pelo método `__repr__()` gerado.

* `compare`: Se `True` (o padrão), este campo é incluído nos métodos de igualdade e comparação gerados (`__eq__()`, `__gt__()`, etc.).

In [ ]:
dir(ItemDeInventario)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'custo_total',
 'quantidade_em_estoque']

In [ ]:
from dataclasses import field
import random

@dataclass(order=True)
class ItemDeInventario:
    '''Classe para registrar um item no inventário.'''
    nome: str = field(compare=False)
    preco_unitario: float = field(compare=False)
    quantidade_em_estoque: int = 0

    def custo_total(self) -> float:
        return self.preco_unitario * self.quantidade_em_estoque

    def __str__(self) -> str:
        return f"{self.nome}: Quantidade - {self.quantidade_em_estoque}, Preço: R${self.preco_unitario:.2f}"

itens = []
for i in range(10):
    itens.append(ItemDeInventario(f"Nome{i:02d}",
                                  random.randint(1,100),
                                  random.randint(100, 1000)))


for item in sorted(itens, reverse=True):
    print(item)

Nome06: Quantidade - 946, Preço: R$93.00
Nome02: Quantidade - 914, Preço: R$39.00
Nome09: Quantidade - 842, Preço: R$100.00
Nome05: Quantidade - 745, Preço: R$65.00
Nome03: Quantidade - 742, Preço: R$62.00
Nome07: Quantidade - 654, Preço: R$72.00
Nome04: Quantidade - 386, Preço: R$19.00
Nome08: Quantidade - 350, Preço: R$89.00
Nome01: Quantidade - 161, Preço: R$24.00
Nome00: Quantidade - 143, Preço: R$76.00


## `Enum`

O conceito de enumeração existe em muitas linguagens. Como definido na própria [documentação do Python sobre Enums](https://docs.python.org/3/library/enum.html):

* É um conjunto de nomes simbólicos (membros) vinculados a **valores constantes**
* Pode ser iterado para retornar seus membros canônicos (ou seja, não alias) **em ordem de definição**

Adicionalmente, a estrutura Enum no Python tem as seguintes características:
* Pode ser criado através da sintaxe de classe ou de função
* Usa sintaxe de chamada para retornar membros por valor
* usa sintaxe de índice para retornar membros por nome

In [ ]:
from enum import Enum

# Criando Enum com sintaxe de classe.
# Usa-se a relação de herança.
class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

# Criando Enum com sintaxe de função.
# Enum (name, values)
Color = Enum('Color', ['RED', 'GREEN', 'BLUE'])

print('\nAcessando os objetos')
print(Color(1)) # sintaxe de chamada
print(Color['RED'], Color.RED) # sintaxe de índice

# acessando o nome e o valor correspondente de um item
print(Color(1).name, Color(1).value)
print(Color['RED'].name, Color['RED'].value)

print('\nIterando no Enum')
for c in Color:
    print(c)

print('\nPor dentro dos objetos')
print(dir(Color))
print(dir(Color.RED))


Acessando os objetos
Color.RED
Color.RED Color.RED
RED 1
RED 1

Iterando no Enum
Color.RED
Color.GREEN
Color.BLUE

Por dentro dos objetos
['BLUE', 'GREEN', 'RED', '__class__', '__doc__', '__members__', '__module__']
['__class__', '__doc__', '__module__', 'name', 'value']


Usar Enums ajuda a evitar erros, garantindo que apenas valores válidos sejam atribuídos, além de tornar o código mais legível e fácil de manter. No exemplo a seguir, a variável `cor` de um objeto `Carro` é uma instância da enumeração `Color`.

In [ ]:
from dataclasses import dataclass

@dataclass
class Carro:
    marca: str
    modelo: str
    cor: Color
    placa: str

carro01 = Carro('Volkswagen', 'T-Cross', Color.RED, "XYZ0000")
carro01.cor, type(carro01.cor)

(<Color.RED: 1>, <enum 'Color'>)

Por padrão, o Enum atribui ao seus elementos um intervalo numérico (`int`) sequencial, com o primeiro valor igual a `1`. Mas a estrutura te dá liberdade de atribuir quaisquer valores, inclusive de tipos heteregêneos (onde cada elemento do Enum tem um tipo). No entanto, tipos heterogêneos devem ser evitados, já que o propósito do Enum é agrupar constantes de características semelhantes.

In [ ]:
# Enum padrão
Color = Enum('Color', ['RED', 'GREEN', 'BLUE'])
print(Color.RED.name, Color.RED.value)

class SwitchPosition(Enum):
    ON = True
    OFF = False

class Size(Enum):
    S = "small"
    M = "medium"
    L = "large"
    XL = "extra large"

# Tipos heterogêneos não recomendado.
# O ideal é haver consistência de tipos
class UserResponse(Enum):
    YES = 1
    NO = "No"

RED 1


A sintaxe funcional permite atributos adicionais. Os mais importantes são:

* `value`: O nome da nova classe de enumeração
* `names`: Nomes para os membros da enumeração
* `start`: Valor inicial que a enumeração de `values` vai começar.
* `type`: Uma classe que adiciona funcionalidade aos elementos do Enum.

> O atributo `type` se expressa na sintaxe de classe como uma herança múltipla, permitindo que os itens da enumeração possam ser comparados com objetos do tipo indicado.


Você pode ler mais sobre nesse tutorial do Real Python: [*Creating Enumerations With the Functional API*](https://realpython.com/python-enum/#creating-enumerations-with-the-functional-api).

Também recomendo ler a seção [*Exploring Other Enumeration Classes*](https://realpython.com/python-enum/#exploring-other-enumeration-classes) desse mesmo tutorial, caso tenha interesse de conhecer outras classes de enumeração (`IntEnum`, `IntFlag`, etc.).

In [ ]:
Size = Enum('Size', ['S', 'M', 'L', 'XL'], start=10, type=int)
print(list(Size))

class Size(int, Enum):
    S = 10
    M = 11
    L = 12
    XL = 13

print(list(Size))
Size.S > 10

[<Size.S: 10>, <Size.M: 11>, <Size.L: 12>, <Size.XL: 13>]
[<Size.S: 10>, <Size.M: 11>, <Size.L: 12>, <Size.XL: 13>]


False

## Pydantic

Segundo a própria [documentação](https://docs.pydantic.dev/latest/), o Pydantic é a biblioteca de **validação de dados** mais utilizada para Python. Adotando a abordagem nativa de anotação de tipos que acabamos de ver, o Pydantic faz o papel de testar o tipo dos objetos associados às variáveis anotadas, **lançando erros** em caso de inconsistências.

Uma outra importante vantagem do Pydantic é a **serialização** de dados, facilmente convertendo o conteúdo dos objetos Pydantic para tipos mais simples de armazenar, como dicionários ou strings JSON.

Para o exemplo que vai ilustrar nossa explicação, precisamos [instalar o Pydantic](https://docs.pydantic.dev/latest/install/) com a dependência opcional de validação de e-mail.

In [ ]:
!pip install pydantic[email]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
from uuid import UUID, uuid4 # universally unique identifiers
from datetime import date
from enum import Enum
import json
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator
)

### `BaseModel`

Similar às dataclasses, os esquemas de dados no Pydantic exploram a estruturação de classes dedicadas ao armazenamento de dados. Ao criar uma classe que herda da classe `BaseModel` do Pydantic, todos os atributos com anotação de tipos serão considerados campos (*fields*) do esquema de dados e, portanto, farão parte das funções de validação e serialização automaticamente adicionadas à classe.

Na célula a seguir, criamos o esquema de dados usado como exemplo no tutorial do Real Python [*Pydantic: Simplifying Data Validation in Python*](https://realpython.com/python-pydantic/). Sua definição envolve duas classe:

* `Department(Enum)`: Um enumerador com valores do tipo string que define os possíveis departamentos de uma empresa hipotética. Será utilizado como o tipo de um dos campos do esquema de dados. O campo internamente irá armazenar strings, porém com a garantia que vai assumir apenas valores válidos.

* `Employee(BaseModel)`: Esquema de dados para o registro de funcionários. Consiste nos atributos id, nome, email, data de nascimento, salário, departamento (nosso enumerador) e uma variável booleano que define se ele é elegível a benefícios.
    * Note que a variável `employee_id` possui um valor default dado pela função `uuid4()`, que cria um id único para o funcionário. É uma prática comum para criação de chaves de um registro em bancos de dados, sem depender de atributos que possam ter duplicatas ou sejam de alguma maneira inconsistentes (sujeito a mudanças, etc.).
    * Já o atributo `email` é do tipo `EmailStr`, um tipo importado da biblioteca Pydantic que possui validações internas de uma string de e-mail. Internamente usa a biblioteca [email-validator](https://pypi.org/project/email-validator/) do Python.
    * Por fim, o atributo `date_of_birth`, por ser do tipo `date` da biblioteca `datetime`, aceita strings formatadas como uma data e as converte em um objeto de data. A classe `date` também possui validações internas para realizar a conversão.


In [ ]:
class Department(Enum):
    HR = "HR"
    SALES = "SALES"
    IT = "IT"
    ENGINEERING = "ENGINEERING"

class Employee(BaseModel):
    employee_id: UUID = uuid4()
    name: str
    email: EmailStr
    date_of_birth: date
    salary: float
    department: Department
    elected_benefits: bool

schema = Employee.model_json_schema()
print(json.dumps(schema, indent=2))

{
  "$defs": {
    "Department": {
      "enum": [
        "HR",
        "SALES",
        "IT",
        "ENGINEERING"
      ],
      "title": "Department",
      "type": "string"
    }
  },
  "properties": {
    "employee_id": {
      "default": "11377385-75ed-4ce3-9518-33ecd9c517a7",
      "format": "uuid",
      "title": "Employee Id",
      "type": "string"
    },
    "name": {
      "title": "Name",
      "type": "string"
    },
    "email": {
      "format": "email",
      "title": "Email",
      "type": "string"
    },
    "date_of_birth": {
      "format": "date",
      "title": "Date Of Birth",
      "type": "string"
    },
    "salary": {
      "title": "Salary",
      "type": "number"
    },
    "department": {
      "$ref": "#/$defs/Department"
    },
    "elected_benefits": {
      "title": "Elected Benefits",
      "type": "boolean"
    }
  },
  "required": [
    "name",
    "email",
    "date_of_birth",
    "salary",
    "department",
    "elected_benefits"
  ],
  "title"

Um registro do esquema é definido como uma instância da classe que herda de `BaseModel`. Na célula a seguir criamos uma instância apenas com os valores obrigatórios (`employee_id` será gerado automaticamente). O Pydantic realizará todo o processo de conversão (coerção), transformando os dados brutos fornecidos em objetos de tipos definidos na classe.

Experimente substituir o objeto instanciado na célula a seguir pelo objeto a seguir. Note como o Pydantic irá apontar todos os campos preenchidos de forma inválida.
```python
new_employee = Employee(
    name=2,
    email="mila.br",
    date_of_birth=1991,
    salary="sim",
    department="EDU",
    elected_benefits=1234,
)
```

In [ ]:
# definindo nova entrada como instância do modelo
new_employee = Employee(
    name="Mila Laranjeira",
    email="mila@projetodesenvolve.com.br",
    date_of_birth="1991-01-01",
    salary=1234.56,
    department="IT",
    elected_benefits=True,
)

new_employee.model_dump()
# new_employee.model_dump_json()

{'employee_id': UUID('11377385-75ed-4ce3-9518-33ecd9c517a7'),
 'name': 'Mila Laranjeira',
 'email': 'mila@projetodesenvolve.com.br',
 'date_of_birth': datetime.date(1991, 1, 1),
 'salary': 1234.56,
 'department': <Department.IT: 'IT'>,
 'elected_benefits': True}

Para reforçar a ideia de "coerção de tipos", pegamos também o exemplo da documentação do Pydantic. Note que o atributo `tastes` define um dicionário `(str, PositiveInt)`, porém o registro possui valores que apesar de incorretos podem ser facilmente convertidos. Da mesma forma, a data fornecida não é um objeto do tipo data, mas é uma string formatada para viabilizar a conversão de tipo.

In [ ]:
from pydantic import PositiveInt

class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: date | None
    tastes: dict[str, PositiveInt]


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01',
    'tastes': {
        'wine': 9,
        b'cheese': 7.0,
        'cabbage': '1',
    },
}

## Ao prefixar um dicionário com o operador **,
## você fornece argumentos de palavra-chave para a
## função com base nos pares de chave-valor do dicionário.
user = User(**external_data)

print(user.id)
print(user.model_dump())

123
{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.date(2019, 6, 1), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}


### Field

Voltando ao exemplo do esquema `Employee`, falaremos agora dos campos (*fields*), também velhos conhecidos da biblioteca `dataclasses`. O field serve o propósito de customizar cada um dos campos bem como suas validações. Segue alguns dos parâmetros interessantes de customização de campos:
* `default` e `default_factory`: assim como em `dataclasses`, podemos definir um valor padrão (`default`) ou um *callable* (função, classe, etc.) que produz um valor padrão (`default_factory`).
* `alias`: Você pode definir um apelido para o objeto, que será usado no processo de validação (entrada de dados) e serialização (saída de dados), podendo ser diferente do nome da variável. Você pode optar também pelos parâmetros mais específicos `validation_alias` ou `serialization_alias`.
* `frozen`: Se `True`, torna o atributo imutável. Tentativas de alterar o valor em uma instância vão lançar um erro.
* `repr`: Se o atributo deve ser incluído em `__repr__` (usado pelos prints e outras saídas formatadas). Pode ser usado em dados sensíveis ou sigilosos.
* `min_length`, `max_length`: Tamanho máximo ou mínimo para iteráveis.
* `gt`, `lt`, `ge`, `le`: Estabelece regras para números, devendo ser maior que | menor que | maior ou igual | menor ou igual que um dado valor.

Veja uma lista detalhada dos parâmetros da classe Field [na documentação](https://docs.pydantic.dev/latest/api/fields/).

> **`ValidationError`** é uma classe customizada pelo Pydantic que herda de exceções do Python. É capaz de capturar erros de validação em objetos de exceção, permitindo sua impressão e tratamento de maneira mais legível e estruturada. Você pode ler mais sobre exceções Pydantic e como customizá-las na [documentação](https://docs.pydantic.dev/latest/errors/errors/).

In [ ]:
from pydantic import ValidationError

class Employee(BaseModel):
    employee_id: UUID = Field(default_factory=uuid4, frozen=True)
    name: str = Field(min_length=2, frozen=True)
    email: EmailStr = Field(pattern=r".+@projetodesenvolve\.com\.br$") # regex - regular expression
    date_of_birth: date = Field(alias="birth_date", repr=False, frozen=True)
    salary: float = Field(alias="compensation", gt=0, repr=False)
    department: Department
    elected_benefits: bool

try:
    employee_data = dict(
        name="Mila Laranjeira",
        email="mila@projetodesenvolve.com.br",
        birth_date="1991-01-01",
        compensation=1234.56,
        department="IT",
        elected_benefits=True,
    )
    employee = Employee(**employee_data)
    print(employee.model_dump())
except ValidationError as e:
    print(json.dumps(e.errors(), indent=2))

{'employee_id': UUID('e1da51a3-d0ca-4952-872b-4002d94b82ac'), 'name': 'Mila Laranjeira', 'email': 'mila@projetodesenvolve.com.br', 'date_of_birth': datetime.date(1991, 1, 1), 'salary': 1234.56, 'department': <Department.IT: 'IT'>, 'elected_benefits': True}


A seguir, instanciamos novos registros com os método de classe `model_validate` e `model_validate_json`. Note que no exemplo anterior tivemos que usar o operador unário `**` para converter o dicionário em argumentos. Esses métodos de `BaseModel` aceitam diretamente argumentos tipo `dict` e `json`, validando cada um dos campos e retornando o objeto instanciado.

In [ ]:
## definindo novo dado como dicionário
employee_data = {
    "name": "Mila Laranjeira",
    "email": "mila@projetodesenvolve.com.br",
    "birth_date": "1991-01-01",  ## chave alias
    "compensation": 1234.56,     ## chave alias
    "department": "IT",
    "elected_benefits": True,
}
employee = Employee.model_validate(employee_data)

new_employee_json = """
 {"employee_id":"d2e7b773-926b-49df-939a-5e98cbb9c9eb",
 "name":"Mila Laranjeira",
 "email":"mila@projetodesenvolve.com.br",
 "birth_date":"1991-01-01",
 "compensation":1234.56,
 "department":"IT",
 "elected_benefits":true}
 """
new_employee = Employee.model_validate_json(new_employee_json)

### `field_validator` e `model_validator`

O tutorial do Real Python cria duas situações hipotéticas para ilustrar o uso de **validadores customizados**, quando queremos impor regras de negócio para além dos tipos de dados.
* Validação de campo: Suponha que a sua empresa apenas contrata maiores de 18 anos. Para trabalhar essa regra, temos o campo data de nascimento, mas por se tratar de um objeto tipo `date`, precisamos criar uma validação customizada para o campo.
* Validação de modelo: Suponha agora que o departamento de TI da empresa é terceirizada e portanto não é elegível a benefícios de funcionários internos. Essa é uma validação que envolve mais de um campo e portanto não é suficiente criar uma validação de campo único.

Ambos `field_validator` e `model_validator` são importados do Pydantic e utilizados como decoradores de função. As representações funcionais a seguir apresentam os parâmetros de cada objeto.

```python
field_validator(field, *fields=(), mode='after', check_fields=None)
```
* `field`: parâmetro obrigatório com o campo a ser validado
* `*fields`: campos adicionais que também devem se sujeitar à mesma validação (independentes um do outro).
* `mode`: se a validação acontece antes ou depois da validação padrão de tipos.
* `check_fields`: se `True`, verifica se os campos existem no modelo.

```python
model_validator(mode)
```
* `mode`: parâmetro obrigatório que define se a validação acontece antes ou depois da validação padrão de tipos.



In [ ]:
class Employee(BaseModel):
    employee_id: UUID = Field(default_factory=uuid4, frozen=True)
    name: str = Field(min_length=1, frozen=True)
    email: EmailStr = Field(pattern=r".+@example\.com$")
    date_of_birth: date = Field(alias="birth_date", repr=False, frozen=True)
    salary: float = Field(alias="compensation", gt=0, repr=False)
    department: Department
    elected_benefits: bool

    @field_validator("date_of_birth")
    @classmethod
    def check_valid_age(cls, date_of_birth: date) -> date:
        today = date.today()
        eighteen_years_ago = date(today.year - 18, today.month, today.day)

        if date_of_birth > eighteen_years_ago:
            raise ValueError("Employees must be at least 18 years old.")

        return date_of_birth

    @model_validator(mode="after")
    def check_it_benefits(self) -> Self:
        department = self.department
        elected_benefits = self.elected_benefits

        if department == Department.IT and elected_benefits:
            raise ValueError(
                "IT employees are contractors and don't qualify for benefits"
            )
        return self


## falha no field validator
try:
    employee_data = {
        "name": "Jake Bar",
        "email": "jbar@example.com",
        "birth_date": '2020-05-21',
        "compensation": 90_000,
        "department": "SALES",
        "elected_benefits": True,
    }
    employee = Employee.model_validate(employee_data)
except ValueError as e:
    print(e)
## falha no model validator
try:
    new_employee = {
        "name": "Alexis Tau",
        "email": "ataue@example.com",
        "birth_date": "2001-04-12",
        "compensation": 100_000,
        "department": "IT",       ### IT
        "elected_benefits": True, ### não recebe benefício
    }

    employee = Employee.model_validate(new_employee)
except ValueError as e:
    print(e)

1 validation error for Employee
birth_date
  Value error, Employees must be at least 18 years old. [type=value_error, input_value='2020-05-21', input_type=str]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error
1 validation error for Employee
  Value error, IT employees are contractors and don't qualify for benefits [type=value_error, input_value={'name': 'Alexis Tau', 'e...elected_benefits': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/value_error
